__This will be where I try to add patches.__

In [13]:
import hoomd,imp
import hoomd
from hoomd import *
from hoomd import md
import numpy as np
import gsd.hoomd
import time 

class SimBox:
    
    ###Creates Initial Snapshot
    def MakeSnapshot(self, boxsize, nparticles): 
        
        snapshot=hoomd.data.make_snapshot(N=nparticles,box=hoomd.data.boxdim(L=boxsize),
                                          bond_types=['tether'],particle_types=self.GenParticleTypes(nparticles, debug=True))
        return snapshot
    
#####--------------------Deprecated--------------------#######
#     def AddParticleTypes(self, nparticles, debug=None):
#         counter=0
#         APTDebug = []
#         #C = nonrigid particle type
#         for k in range(nparticles):
#             counter=k+1
#             system.particles.types.add('C%d'%counter)
#             APTDebug.append('C%d'%counter)
#         if(debug==True):
#             print("DEBUG: AddParticleRypes: Particle Types = ",APTDebug)
#         return APTDebug
#####--------------------Deprecated--------------------#######

    def GenParticleTypes(self, nparticles, debug=None):
        counter=0
        #List that will contain the particle types. Enable debug to print.
        GPT = [] 
        for k in range(nparticles):
            counter=k+1
            GPT.append('C%d'%counter)
        if(debug==True):
            print("DEBUG: GenParticleTypes: Particle Types = ",GPT)
        return GPT
            
#####--------------------Deprecated--------------------#######
#     def PatchTypes(self, nrigid, npatch, debug=None):
#         patch_types = []
#         for i in range(npatch*nrigid):
#             patch_types.append(system.particles.types[nrigid+i])
#         if(debug==True):
#             print("DEBUG: PatchTypes: Patch Types = ", patch_types)
#         return patch_types
#####--------------------Deprecated--------------------#######    
#     def PatchTypes(self, tot_npatch, debug=None):
#         patch_types = []
#         for i in range(tot_npatch):
#             patch_types.append('%d'%i)
#         if(debug==True):
#             print(patch_types)
#         return patch_types
   
#     def ptags(self, nrigid, npatch, nparticles, debug=None):
#         x = nrigid + npatch*nrigid + nparticles
#         if(debug==True):
#             print("DEBUG: ptags: Particle Tage = ", np.linspace(0,x-1,x))
#             return np.linspace(0,x-1,x)
#         else:
#             return np.linspace(0,x-1,x) 
        


In [23]:
class Polymer:
    ###To Do:
        #user input for k and r0
        #Define the bonds between particles
            #just have it be linear with rigids on either end at first
#     def MakePolyParticles(self, nparticles, types, debug=None):
#         index = nrigid + npatch*nrigid
#         MakePolyDebug = []
#         for i in range(nparticles):
#             system.particles.add(types[index + i])
#             #system.particles.get(index+i).position = [0 + i, i, 0]
#             system.particles.get(index+i).position = [-8 + i, 0, 0]
#             system.particles.get(index+i).diameter = 0
#             system.particles.get(index+i).moment_intertia = [0,0,0]
#             MakePolyDebug.append([-8 + i, 0, 0])
#         if(debug==True):
#             print("DEBUG: MakePolyParticles: Polymer Particle Positions = ", MakePolyDebug)

    def SetPolyProperties(self, nparticles, boxsize, diam=None, debug=None):
        if(diam==None):
            diam=1 #default diameter
        for i in range(nparticles):
            system.particles.get(i).position = [nparticles+i,0,0]
            system.particles.get(i).moment_intertia = [0,0,0]
            system.particles.get(i).diameter = diam
            system.particles.get(i).type=('C%d'%(i+1))
            if(debug==True):
                print("DEBUG: SetPolyProperties: Particle %d is at "%i, [nparticles+i,0,0])
            
#     def PatchTypes(self, tot_npatch, debug=None):
#         patch_types = []
#         for i in range(tot_npatch):
#             system.particles.types.add('%d'%(i+1))
#             system.particles.add('%d'%(i+1))
#         if(debug==True):
#             print("DEBUG: PatchTypes: ",patch_types)
#         return patch_types
    
    def SetRigidProperties(self, index, moment, diam):
            system.particles.get(index).moment_intertia = moment
            system.particles.get(index).diameter = diam
            
            
    def RigidParam(self, rigid_type, patch_positions, patch_types, pdiam=None):
        rigid.set_param(rigid_type,
                            positions=patch_positions, 
                            types=patch_types,
                            #diameters=pdiam
                           )
    
    def MakeBonds(self, nparticles, debug=None):
        DebugBonds = []
        f=0
        for f in range(nparticles-1):
            system.bonds.add('tether',f,f+1)
            DebugBonds.append([f,f+1])
            f+=1
        if(debug==True):
            print("DEBUG: MakeBonds: Bonds = ", DebugBonds)  

In [24]:
class Debug:
    def PrintAllParticleTags(self, nparticles, tot_npatch):
        X = []
        for i in range(nparticles+tot_npatch):
            X.append(system.particles[i].tag)
        print("DEBUG: List of All Particle Tags: ", X)

In [48]:
import hoomd,imp
import hoomd
from hoomd import *
from hoomd import md
import numpy as np
import gsd.hoomd
import time 

#Number of rigid bodies
#nrigid = 2
#Box Size
boxsize = 100
#Number of constituent particles
nparticles = 10
#Number of Patches per Rigid Body
npatch=1
#Total number of patches in sim
tot_npatch = 1
#Rigid Diameter
diam = 1
#List of Patch Diameters
pdiam = [0.5]
#Angle of patches
angle = 30*np.pi/180
##----------SETUP----------##
context.initialize("")

###Placeholers for sig and eps values
# sig = [1]*(nparticles+tot_npatch)
# eps = [0.1]*(nparticles+tot_npatch)
sig = [1.0]*(nparticles)
eps = [0.5]*(nparticles)


print("sig = ", sig)
print("eps = ", eps)

DB = Debug()
SB = SimBox()
Poly = Polymer()

snapshot = SB.MakeSnapshot(boxsize, nparticles)

system = hoomd.init.read_snapshot(snapshot)

Poly.MakeBonds(nparticles, debug=True)

##Adds patch types to system based on total number of patches
#Poly.PatchTypes(tot_npatch, debug=True)



#system.particles.types.add('1')



# system.particles.add('1')
#types = SB.GenParticleTypes(nparticles)


types = system.particles.types
print("DEBUG: types: ",types)


Poly.SetPolyProperties(nparticles, boxsize, diam=0.9, debug=True)


#RigidParam(rigid_type, patch_positions, patch_types, pdiam, npatch)
#Poly.RigidParam('C1',[99*np.cos(angle),np.sin(angle),0],['0'])
# x=99*np.cos(angle)
# y=np.sin(angle)
# z=0
#################################
#rigpos = system.particles[0].position

#pos = [(5+0.5)*np.cos(angle),(0+0.5)*np.sin(angle),0]

#Debug Commandi

#rigid=hoomd.md.constrain.rigid()

#rigid.set_param('C1',positions=[[5-np.cos(angle),np.sin(angle),0]], types=['1'], diameters = pdiam)
#rigid.set_param('C1',positions=[pos], types=['1'], diameters = pdiam)

#rigid.create_bodies()
#DB.PrintAllParticleTags(nparticles, tot_npatch)
#Defines harmonic bonds
harm = md.bond.harmonic()
harm.bond_coeff.set('tether',k=20,r0=1.0)

#Groups rigid centers
#center = hoomd.group.rigid_center()

#Groups all particles
part = hoomd.group.all()

#LJ Cutoff
rcut = 1.0

#Neighbor List
nl = md.nlist.cell(check_period = 1)


#Defines LJ
lj = md.pair.lj(r_cut=rcut, nlist=nl)

#Calculates LJ forces (this might be causing problems)
for i in range(0,len(sig)):
    for j in range(i,len(sig)):
        lj.pair_coeff.set(types[i],types[j],epsilon=0.5*(eps[i]+eps[j]),sigma=0.5*(sig[i]+sig[j]))
        
        #Integrators. 
hoomd.md.integrate.mode_standard(dt=0.002)
hoomd.md.integrate.langevin(group=part, kT=0.1, seed = 5)
# hoomd.md.integrate.langevin(group=hoomd.group.nonrigid(), kT=0.01, seed = 5)
# hoomd.md.integrate.langevin(group=center, kT=0.01, seed = 5)

#Allows me to make unique file names
import time 
t = time.strftime(time.strftime("%d %b %H:%M:%S", time.gmtime()))

### DUMP AND RUN ###
hoomd.analyze.log(filename='random.log',quantities=['temperature','num_particles','ndof',
                                                    'translational_ndof','rotational_ndof',
                                                    'potential_energy'],period=1000,header_prefix='#'
                                                      ,overwrite=True)
#hoomd.deprecated.dump.xml(group=part,filename ='random.xml',vis=True,image=True)
hoomd.dump.gsd(filename='random %s.gsd'%t,period=1000, group = hoomd.group.all() ,overwrite = True)
hoomd.run(1e5)

sig =  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
eps =  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
DEBUG: GenParticleTypes: Particle Types =  ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10']
notice(2): Group "all" created containing 10 particles
DEBUG: MakeBonds: Bonds =  [[0, 1], [1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7], [7, 8], [8, 9]]
DEBUG: types:  Particle types: ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10']
DEBUG: SetPolyProperties: Particle 0 is at  [-10, 0, 0]
DEBUG: SetPolyProperties: Particle 1 is at  [-9, 0, 0]
DEBUG: SetPolyProperties: Particle 2 is at  [-8, 0, 0]
DEBUG: SetPolyProperties: Particle 3 is at  [-7, 0, 0]
DEBUG: SetPolyProperties: Particle 4 is at  [-6, 0, 0]
DEBUG: SetPolyProperties: Particle 5 is at  [-5, 0, 0]
DEBUG: SetPolyProperties: Particle 6 is at  [-4, 0, 0]
DEBUG: SetPolyProperties: Particle 7 is at  [-3, 0, 0]
DEBUG: SetPolyProperties: Particle 8 is at  [-2, 0, 0]
DEBUG: SetPolyProperties: Particle 

In [17]:
print(system.particles)

Particle Data for 7 particles of 6 type(s)


In [18]:
for i in range(nparticles+tot_npatch+2):
    print(system.particles[i].tag)

0
1
2
3
4
5
6


IndexError: 

In [16]:
print(types)

Particle types: ['C1', 'C2', 'C3', 'C4', 'C5', '1']


In [14]:
for i in range(0,len(sig)):
    for j in range(i,len(sig)):
        print([types[i],types[j]])
        
        
        
        
        
        

['C1', 'C1']
['C1', 'C2']
['C1', 'C3']
['C1', 'C4']
['C1', 'C5']
['C1', '1']
['C2', 'C2']
['C2', 'C3']
['C2', 'C4']
['C2', 'C5']
['C2', '1']
['C3', 'C3']
['C3', 'C4']
['C3', 'C5']
['C3', '1']
['C4', 'C4']
['C4', 'C5']
['C4', '1']
['C5', 'C5']
['C5', '1']
['1', '1']


In [13]:
for i in range(7):
    print(system.particles[i])

tag         : 0
position    : (-5.9075350424352, -0.2861289021940368, 1.8751983218924329)
image       : (0, 0, 0)
velocity    : (0.005078939358090133, -0.020003050289566005, 0.06572111829247466)
acceleration: (-2.377257676690709, -3.3456095840313798, 3.7884934459454196)
charge      : 0.0
mass        : 1.0
diameter    : 0.9
type        : C1
typeid      : 0
body        : 0
orientation : (1.0, 0.0, 0.0, 0.0)
mom. inertia: (0.0, 0.0, 0.0)
angular_momentum: (0.0, 0.0, 0.0, 0.0)
net_force   : (0.19351273654908996, -0.004863745568864689, 0.007162475140636997)
net_energy  : -0.03254059311030808
net_torque  : (0.0, 0.0, 0.0)
net_virial  : (-0.12519421939820227, 0.0032111715945809098, -0.0050111812094450845, 0.00034627812798008204, -0.002377714662859695, 0.014453294359972082)

tag         : 1
position    : (-4.581334703521241, -0.10570939905684144, 0.6744894396432881)
image       : (0, 0, 0)
velocity    : (-0.11590605153054807, -0.049511545766150994, -0.09218474817839015)
acceleration: (0.755328

In [38]:
print(system.particles[1])

tag         : 1
position    : (-2.0, 0.0, 0.0)
image       : (1, 0, 0)
velocity    : (0.0, 0.0, 0.0)
acceleration: (0.0, 0.0, 0.0)
charge      : 0.0
mass        : 1.0
diameter    : 0.9
type        : C1
typeid      : 0
body        : 4294967295
orientation : (1.0, 0.0, 0.0, 0.0)
mom. inertia: (0.0, 0.0, 0.0)
angular_momentum: (0.0, 0.0, 0.0, 0.0)
net_force   : (0.0, 0.0, 0.0)
net_energy  : 0.0
net_torque  : (0.0, 0.0, 0.0)
net_virial  : (0.0, 0.0, 0.0, 0.0, 0.0, 0.0)

